In [39]:
from os import walk
import pandas as pd
import numpy as np

cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

f = []
for (dirpath, dirnames, filenames) in walk('../blend/'):
    f.extend(filenames)
    break

ff = []
for (dirpath, dirnames, filenames) in walk('../skolbachev/'):
    for d in dirnames:
        for (dirpath, dirnames2, filenames) in walk('../skolbachev/'+d):
            for qf in filenames:
                ff.append('../skolbachev/'+d+'/'+qf)

train_idx = pd.read_csv('../input/train.csv')['id'].values
train = pd.read_csv('../input/train.csv').sort_values('id').reset_index(drop=True)

In [40]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import minmax_scale
oofs = []
onms = []

train_files = [q for q in f if q.startswith('train')]
for q in train_files:
    nm = q[6:-4]
    nf = pd.read_csv('../blend/'+q).sort_values('id').reset_index(drop=True)
    for c in cols:
        nf[c] = minmax_scale(nf[c])
        #print(nm,c,roc_auc_score(train[c],nf[c]))
    nf.columns = ['id'] + [nm+'_' + q for q in cols]
    print(nm, roc_auc_score(train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate']],nf[[nm+'_toxic',nm+'_severe_toxic',nm+'_obscene',nm+'_threat',nm+'_insult',nm+'_identity_hate']]))
    onms.append(nm)
    oofs.append(nf)
    
sk_train = [q for q in ff if not q.endswith('test_X_pred.npy')]
suf = 'sk'
i = 0
for q in sk_train:
    nf = pd.DataFrame(np.load(q))
    nm = suf+str(i)
    nf.columns = [nm+'_'+q for q in cols]
    nf['id'] = train_idx
    nf = nf.sort_values('id').reset_index(drop=True)
    for c in cols:
        nf[nm+'_'+c] = minmax_scale(nf[nm+'_'+c])
    print(nm, roc_auc_score(train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate']],nf[[nm+'_toxic',nm+'_severe_toxic',nm+'_obscene',nm+'_threat',nm+'_insult',nm+'_identity_hate']]))
    onms.append(nm)
    oofs.append(nf)
    i = i + 1

_att 0.9887701275576107
2ycnn 0.9873606598169475
4gru 0.9876478836087786
5cnn 0.9852476095999143
att 0.9880287618175224
cnnb 0.9852844588080257
grucnn 0.9895349799724892
grucnn_fl 0.9880106712925772
grucnn_fl3 0.9897685069751564
grucnn_fl4 0.9899923857692398
lr 0.9858536442716596
nbsvm 0.9820123460744199
wordbatch2 0.9865481886607744
ycnn 0.9847277643406013
sk0 0.9910875692487013
sk1 0.9911913850256058
sk2 0.9908130451768224
sk3 0.9899530158503688
sk4 0.9905625848095353
sk5 0.9902882130480376
sk6 0.990813388694949
sk7 0.9904216084142267
sk8 0.9911659724246551
sk9 0.9905731062720363
sk10 0.9907451200741724


In [41]:
train = pd.read_csv('../input/train.csv').sort_values('id').reset_index(drop=True)
for o in oofs:
    train = train.merge(o, on='id', how='left')

In [42]:
onms

['_att',
 '2ycnn',
 '4gru',
 '5cnn',
 'att',
 'cnnb',
 'grucnn',
 'grucnn_fl',
 'grucnn_fl3',
 'grucnn_fl4',
 'lr',
 'nbsvm',
 'wordbatch2',
 'ycnn',
 'sk0',
 'sk1',
 'sk2',
 'sk3',
 'sk4',
 'sk5',
 'sk6',
 'sk7',
 'sk8',
 'sk9',
 'sk10']

In [43]:
train.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate', '_att_toxic', '_att_severe_toxic',
       ...
       'sk9_obscene', 'sk9_threat', 'sk9_insult', 'sk9_identity_hate',
       'sk10_toxic', 'sk10_severe_toxic', 'sk10_obscene', 'sk10_threat',
       'sk10_insult', 'sk10_identity_hate'],
      dtype='object', length=158)

In [44]:
orig_cols = ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [45]:
scores = {}
for c in cols:
    y = train[c]
    q = train.drop(orig_cols, axis=1)
    scores[c] = []
    for n in onms:
        w = roc_auc_score(y,train[n+'_'+c])
        scores[c].append(w)
        #print(n,w)

In [46]:
from sklearn.metrics import log_loss
p = []
for c in cols:
    ws = scores[c]
    y = train[c]
    pred = 0
    i = 0
    for n in onms:
        pred += train[n+'_'+c]
    p.append(roc_auc_score(y,minmax_scale(pred)))
    print(c,roc_auc_score(y,minmax_scale(pred)))
np.mean(p)

toxic 0.9888762745435871
severe_toxic 0.992433472296018
obscene 0.9954761589214753
threat 0.9954818406128444
insult 0.9903930486907487
identity_hate 0.9918056541613584


0.9924110748710052

In [11]:
from sklearn.metrics import log_loss
p = []
for c in cols:
    ws = scores[c]
    y = train[c]
    pred = 0
    i = 0
    for n in onms:
        pred += ((ws[i]-np.min(ws))/(np.max(ws)-np.min(ws))+0.1)*train[n+'_'+c]
    p.append(roc_auc_score(y,minmax_scale(pred)))
    print(c,roc_auc_score(y,minmax_scale(pred)))
np.mean(p)

toxic 0.9887502736948446
severe_toxic 0.9923729893180075
obscene 0.9952371552537522
threat 0.9952858814429402
insult 0.9903213795502691
identity_hate 0.9910213977179613


0.9921648461629625

In [50]:
preds = []

train_files = [q for q in f if q.startswith('test')]
for q in train_files:
    nf = pd.read_csv('../blend/'+q)
    for c in cols:
        nf[c] = minmax_scale(nf[c])
    preds.append(nf)
    
sk_train = [q for q in ff if q.endswith('test_X_pred.npy')]
suf = 'sk'
i = 0
for q in sk_train:
    nf = pd.DataFrame(np.mean(np.load(q),axis=0))
    nf.columns = cols
    for c in cols:
        nf[c] = minmax_scale(nf[c])
    preds.append(nf)
    i = i + 1

In [51]:
onms

['_att',
 '2ycnn',
 '4gru',
 '5cnn',
 'att',
 'cnnb',
 'grucnn',
 'grucnn_fl',
 'grucnn_fl3',
 'grucnn_fl4',
 'lr',
 'nbsvm',
 'wordbatch2',
 'ycnn',
 'sk0',
 'sk1',
 'sk2',
 'sk3',
 'sk4',
 'sk5',
 'sk6',
 'sk7',
 'sk8',
 'sk9',
 'sk10']

In [76]:
p = []
for c in cols:
    ws = scores[c]
    y = train[c]
    pred = 0
    i = 0
    for n in onms:
        pred += ((ws[i]-np.min(ws))/(np.max(ws)-np.min(ws))+0.1)*train[n+'_'+c]
        i = i + 1
    p.append(roc_auc_score(y,pred))
    print(c,roc_auc_score(y,pred))
np.mean(p)

toxic 0.9889388979125824
severe_toxic 0.9924436758220326
obscene 0.9955034460357016
threat 0.9955527709418247
insult 0.9904630565955026
identity_hate 0.9919069352020486


0.992468130418282

In [56]:
len(preds)

25

In [57]:
len(ws)

25

In [58]:
sub = pd.read_csv('../input/sample_submission.csv')
for c in cols:
    sub[c] = 0
    ws = scores[c]
    for j in range(len(ws)):
        sub[c] += ((ws[j]-np.min(ws))/(np.max(ws)-np.min(ws))+0.1) * preds[j][c]
    sub[c] = minmax_scale(sub[c])
sub.head(n=3)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.981775,0.442928,0.954507,0.162944,0.932912,0.553202
1,0000247867823ef7,0.005639,0.004155,0.003097,0.002642,0.003744,0.004337
2,00013b17ad220c46,0.006603,0.003422,0.004544,0.003536,0.003556,0.004095


In [59]:
sub.to_csv('weighted_blend_25models.csv', index=False)

In [62]:
fof = []
for (dirpath, dirnames, filenames) in walk('../nooof/'):
    fof.extend(filenames)
    break
len(fof)    

17

In [63]:
sub = pd.read_csv('../input/sample_submission.csv')
for c in cols:
    sub[c] = 0
    for j in range(len(fof)):
        tm = pd.read_csv('../nooof/'+fof[j])
        sub[c] += minmax_scale(tm[c])
    sub[c] = minmax_scale(sub[c])
sub.head(n=3)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.956421,0.383945,0.936247,0.152043,0.908394,0.448139
1,0000247867823ef7,0.037360,0.035600,0.034143,0.031197,0.035935,0.031023
2,00013b17ad220c46,0.039175,0.039903,0.037541,0.031671,0.031913,0.033962


In [64]:
sub.to_csv('all_no_oof_models_blend.csv', index=False)

In [135]:
sub1 = pd.read_csv('all_no_oof_models_blend.csv')
sub2 = pd.read_csv('weighted_blend_21models.csv')

In [136]:
q = pd.DataFrame()
q['toxic_oof'] = sub2.toxic
q['toxic_nooof'] = sub1.toxic
q.corr()

,toxic_oof,toxic_nooof
toxic_oof,1.000000,0.990209
toxic_nooof,0.990209,1.000000


In [137]:
q = pd.DataFrame()
q['toxic_oof'] = sub2.severe_toxic
q['toxic_nooof'] = sub1.severe_toxic
q.corr()

,toxic_oof,toxic_nooof
toxic_oof,1.000000,0.968748
toxic_nooof,0.968748,1.000000


In [142]:
q.corr()['toxic_oof'][1]

0.9687477351009827

In [145]:
for c in cols:
    q = pd.DataFrame()
    q['toxic_oof'] = preds[17][c]
    q['toxic_nooof'] = preds[18][c]
    print(c, q.corr()['toxic_oof'][1])

toxic 0.9912796513988811
severe_toxic 0.9799256447811807
obscene 0.9912714161127426
threat 0.9735262656618802
insult 0.9861293153974795
identity_hate 0.9709932081423762


In [65]:
sub1 = pd.read_csv("weighted_blend_25models.csv")
sub2 = pd.read_csv("all_no_oof_models_blend.csv")
sub3 = pd.read_csv("blend_32_sol.csv")

b1 = sub1.copy()
col = sub1.columns

col = col.tolist()
col.remove('id')

for i in col:
    b1[i] = minmax_scale(minmax_scale(sub1[i]) + minmax_scale(sub2[i]) + minmax_scale(sub3[i]))
    
b1.to_csv('blend_oof2_nooof_all.csv', index = False)

In [23]:
sub1 = pd.read_csv('mean__crawl_300d_0123__cleaned_lstm_03__glove_300d_0123__glove_twitter_200d_0__emb_drop05__bs256__10f__0.03518_0.99215.csv')
sub2 = pd.read_csv('weighted_blend_22models.csv')
sub1['toxic'] = sub2['toxic']
sub1.to_csv('toxic_fixed_in_69model.csv',index=False)

In [31]:
sub1 = pd.read_csv('../blend/train_lr.csv')
sub1.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,prediction_toxic,prediction_severe_toxic,prediction_obscene,prediction_threat,prediction_insult,prediction_identity_hate
0,0000997932d777bf,0,0,0,0,0,0,0.004372,0.007445,0.013798,0.002682,0.011628,0.006198
1,000103f0d9cfb60f,0,0,0,0,0,0,0.011266,0.024178,0.027765,0.005054,0.035209,0.008024
2,000113f07ec002fd,0,0,0,0,0,0,0.016494,0.007536,0.042107,0.002213,0.022258,0.002235
3,0001b41b1c6bb37e,0,0,0,0,0,0,0.001094,0.003275,0.007093,0.002568,0.002749,0.002481
4,0001d958c54c6e35,0,0,0,0,0,0,0.031645,0.042135,0.214759,0.004717,0.122849,0.044512


In [32]:
sub1.drop(cols,axis=1,inplace=True)
sub1.head()

,id,prediction_toxic,prediction_severe_toxic,prediction_obscene,prediction_threat,prediction_insult,prediction_identity_hate
0,0000997932d777bf,0.004372,0.007445,0.013798,0.002682,0.011628,0.006198
1,000103f0d9cfb60f,0.011266,0.024178,0.027765,0.005054,0.035209,0.008024
2,000113f07ec002fd,0.016494,0.007536,0.042107,0.002213,0.022258,0.002235
3,0001b41b1c6bb37e,0.001094,0.003275,0.007093,0.002568,0.002749,0.002481
4,0001d958c54c6e35,0.031645,0.042135,0.214759,0.004717,0.122849,0.044512


In [33]:
sub1.columns = ['id'] + cols
sub1.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,0.004372,0.007445,0.013798,0.002682,0.011628,0.006198
1,000103f0d9cfb60f,0.011266,0.024178,0.027765,0.005054,0.035209,0.008024
2,000113f07ec002fd,0.016494,0.007536,0.042107,0.002213,0.022258,0.002235
3,0001b41b1c6bb37e,0.001094,0.003275,0.007093,0.002568,0.002749,0.002481
4,0001d958c54c6e35,0.031645,0.042135,0.214759,0.004717,0.122849,0.044512


In [34]:
sub1.to_csv('../blend/train_lr.csv',index=False)